<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import math
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from scipy import stats as st
from numpy.random import RandomState
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Зафиксируем seed
np.random.seed(0)

In [3]:
data=pd.read_csv('insurance.csv')

In [4]:
data.sample(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
398,0,30.0,37700.0,1,0
3833,1,29.0,43700.0,1,0
4836,1,38.0,43600.0,1,0
4572,1,27.0,43400.0,1,0
636,0,26.0,36100.0,1,0
2545,0,44.0,38800.0,0,1
1161,0,19.0,39100.0,3,0
2230,1,31.0,35400.0,2,0
148,0,28.0,39000.0,1,0
2530,1,24.0,43500.0,1,0


Посмотрим наличие пропусков:

In [5]:
data.isnull().mean()

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

Пропусков нет

Посмотрим наличие дубликатов:

In [6]:
data.duplicated().sum()

153

In [7]:
#Удаляием полные дубликаты
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

In [8]:
#Дубликатов больше нет.

Посмотрим данные методом describe:

In [9]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811842,1.203425,0.152259
std,0.500049,8.487995,9972.953985,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [10]:
data['Пол'].value_counts()

0    2431
1    2416
Name: Пол, dtype: int64

Поделим колонки на признаки и целевые признаки:

In [11]:
features=data.drop(['Страховые выплаты'],axis=1)
target = data['Страховые выплаты']
print('Размер таблицы с признаками:{}'.format(features.shape))
print('Размер таблицы с целевыми признаками:{}'.format(data.shape))

Размер таблицы с признаками:(4847, 4)
Размер таблицы с целевыми признаками:(4847, 5)


Данные выглядят адекватно

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$
X(X^T X)^{-1} X^T y = XA((XA)^T (XA))^{-1} (XA)^T y
$$

Найдем обратимую матрицу признаков:

**Ответ:** Предсказания не изменились

**Обоснование:** Новые веса линейной регрессии: $w1 = ((XA)^T (XA)^T)^{-1} (XA)^T y$,
где A - обратимая квадратная матрица.

**Доказательство:**

$w1 = ((XA)^T (XA))^{-1} (XA)^Ty $<br>
$ w1 = (A^TX^TXA)^{-1}A^T X^T $y<br>

Вынести за скобки $X^TX$ мы не можем, так как эти матрицы по отдельности не квадратные (они размером число строк х число столбцов признаков, прямоугольная матрица) . Поэтому вынесем за скобки обратимые матрицы, т.е. квадратные:

$w1 = A^{-1}((X^TX)^{-1}(A^T)^{-1}A^TX^Ty)$<br>
$w1 = A^{-1}((X^TX)^{-1}EX^Ty)$<br>
$w1 = A^{-1}((X^TX)^{-1}X^Ty)$<br>
$w1=A^{-1}w$ <br><br>
 **Предсказания:**<br><br>
 $a1=(XA)w1=XAA^{-1}w=Xw=a$

## Алгоритм преобразования

**Алгоритм**

1.Создаем случайную квадратную матрицу размером features.<br>
2.Проверим ее на обратимость.<br>
3.Умножим эту матрицу на исходную матрицу с features.<br>
4.Загрузим эту матрицу на вход в Линейную регрессию и кросс валидацией найдем r2.<br>
5.Тоже сделаем для немодифицированного features.<br>
6.Сравним результаты.

**Обоснование**

Качество линейной регрессии не изменится, так как из за доказанного в п.2 математического выражения, измененные признаки компенсируются другими весами.

## Проверка алгоритма

In [12]:
A=np.random.random((features.shape[1], features.shape[1]))
print(A)
print(np.linalg.inv(A))
#Обратная матрица существует
features_mdfd=features@A
print(features_mdfd.shape)


[[0.76992212 0.4706232  0.92019349 0.0291138 ]
 [0.11528118 0.37601733 0.08510591 0.08788788]
 [0.56687268 0.21612202 0.62653521 0.92939982]
 [0.82388221 0.24843039 0.08966293 0.95019247]]
[[ 0.95483357 -1.36124252 -1.43169781  1.49702047]
 [-0.1957023   3.0811444  -0.10601445 -0.17529902]
 [ 0.41372277 -0.45007189  1.21559212 -1.16003895]
 [-0.81577955  0.41718744  1.15439141 -0.09030428]]
(4847, 4)


In [13]:
%%time
model = LinearRegression()
scores=cross_val_score(model, features, target, cv=5)#, #scoring='r2'#)
final_score= sum(scores)/len(scores)
print('Средняя оценка качества модели:', final_score)

Средняя оценка качества модели: 0.42779425802804977
Wall time: 25.9 ms


In [14]:
%%time
model = LinearRegression()
scores=cross_val_score(model, features_mdfd, target, cv=5)#, #scoring='r2'#)
final_score= sum(scores)/len(scores)
print('Средняя оценка качества модели:', final_score)

Средняя оценка качества модели: 0.4277942580281656
Wall time: 19.9 ms


Качество моделей одинаково до 12 знака после запятой. Метрика линейной регрессии r2, поэтому отдельно ее не пишем.